# **Proyecto Final Herramientas de BigData**

**Maestria en Analitica Aplicada**

**Universidad de La Sabana, Facultad de Ingeniería**


Presentado por: **Julián Andrés Santos Méndez**

email: ingjuliansantos@gmail.com

Archivo: Adquisición de datos con la API de XM S.A. E.S.P.

In [40]:
import pandas as pd
import datetime as dt
import calendar
import requests
import json

## **Extracción de datos de la REST API de XM S.A. E.S.P.**
---

En 2020, XM S.A. E.S.P. puso a disposición del público su API REST para facilitar la consulta de datos del SIN.

Para simplificar aún más el uso de la API, desarrollaron un módulo llamado *pydataxm.py*, que permite realizar consultas a la API de manera más directa y sencilla, devolviendo un *DataFrame* de Pandas con los datos de cada variable consultada.

In [41]:
import pydataxm as api_xm
consult = api_xm.ReadDB()

## Funciones de Apoyo

La API REST de XM tiene un límite máximo de 31 días (registros) por consulta. Para evitar errores al superar este límite, se han creado las siguientes funciones, las cuales permiten descargar un mes a la vez.

In [38]:
def generate_date_dict(start_year, end_year, end_month):
    date_dict = {}
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            if year == end_year and month > end_month:
                break
            start_date = dt.date(year, month, 1)
            end_date = dt.date(year, month, calendar.monthrange(year, month)[1])
            date_dict[(year, month)] = {"start_date": start_date, "end_date": end_date}
    return date_dict

def extract_metric(date_dict, metric, num_metric):
    df_to_concat = pd.DataFrame()
    for key in date_dict:
        start_date = date_dict[key]['start_date']
        end_date = date_dict[key]['end_date']
        print(f"Start for: {key}: start_date = {date_dict[key]['start_date']}, end_date = {date_dict[key]['end_date']}")
        df = consult.request_data(metric, num_metric, start_date, end_date)
        print(f"Done for: {key}: start_date = {date_dict[key]['start_date']}, end_date = {date_dict[key]['end_date']}")
        df_to_concat = pd.concat([df_to_concat, df], ignore_index=True)
    return df_to_concat

## Extracción de Listados

### Listado de Metricas

In [18]:
url = "http://servapibi.xm.com.co/lists"
request = {
    "MetricId": "ListadoMetricas",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_metricas = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_metricas

,Id,Values.MetricId,Values.MetricName,Values.Entity,Values.MaxDays,Values.Type,Values.Url,Values.Filter,Values.MetricUnits,Values.MetricDescription
0,Sistema,DemaReal,Demanda Real por Sistema,Sistema,31,HourlyEntities,http://servapibi.xm.com.co/hourly,No aplica,kWh,Demanda de usuarios regulados y no regulados q...
1,Sistema,DemaReal,Demanda Real por Agente,Agente,31,HourlyEntities,http://servapibi.xm.com.co/hourly,Codigo Comercializador,kWh,Demanda de usuarios regulados y no regulados q...
2,Sistema,ExpoMoneda,Exportaciones Moneda por Sistema,Sistema,31,HourlyEntities,http://servapibi.xm.com.co/hourly,No aplica,COP,Transferencias de Energia desde Colombia hacia...
3,Sistema,DemaCome,Demanda Comercial por Sistema,Sistema,31,HourlyEntities,http://servapibi.xm.com.co/hourly,No aplica,kWh,Considera la demanda propia de cada comerciali...
4,Sistema,Gene,Generación por Sistema,Sistema,31,HourlyEntities,http://servapibi.xm.com.co/hourly,No aplica,kWh,Generacion neta de cada una de las plantas Nac...
...,...,...,...,...,...,...,...,...,...,...
182,Sistema,ListadoAgentes,Listado de agentes con atributos por Sistema,Sistema,731,ListsEntities,http://servapibi.xm.com.co/list,No aplica,,Listado de Agentes registrados ante el ASIC co...
183,Sistema,ListadoRios,Listado de rios por Sistema,Sistema,731,ListsEntities,http://servapibi.xm.com.co/list,No aplica,,Listado de ríos que se encuentran registrados ...
184,Sistema,ListadoEmbalses,Listado de embalses por Sistema,Sistema,731,ListsEntities,http://servapibi.xm.com.co/list,No aplica,,Listado de embalses que se encuentran registra...
185,Sistema,ListadoMetricas,Listado de Métricas por Sistema,Sistema,731,ListsEntities,http://servapibi.xm.com.co/list,No aplica,,Listado de métricas disponibles en la API XM c...


In [19]:
listado_metricas.to_json('./data/listado_metricas.json', orient='records', lines=True)

### Listado de Plantas de Generación

In [20]:
request = {
    "MetricId": "ListadoRecursos",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_plantas = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_plantas

,Id,Values.Code,Values.Name,Values.Type,Values.Disp,Values.RecType,Values.CompanyCode,Values.EnerSource,Values.OperStartdate,Values.State
0,Sistema,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,FILO DE AGUA,ISGG,AGUA,2014-03-31,OPERACION
1,Sistema,2QEK,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION
2,Sistema,2QRL,LA REBUSCA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,GEN. DISTRIBUIDA,HZEG,AGUA,2014-07-24,OPERACION
3,Sistema,2QV2,BAJO TULUA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,EPSG,AGUA,2015-01-30,OPERACION
4,Sistema,2R22,LAGUNETA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,ENDG,AGUA,2014-12-17,OPERACION
...,...,...,...,...,...,...,...,...,...,...
680,Sistema,VNTB,VENTANA B,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,HDTG,AGUA,1957-11-01,OPERACION
681,Sistema,ZPA2,ZIPAEMG 2,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1964-01-01,OPERACION
682,Sistema,ZPA3,ZIPAEMG 3,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1976-01-01,OPERACION
683,Sistema,ZPA4,ZIPAEMG 4,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1981-04-01,OPERACION


In [21]:
listado_plantas.to_json('./data/listado_recursos_plantas.json', orient='records', lines=True)

### Listado de Ríos

In [22]:
request = {
    "MetricId": "ListadoRios",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_rios = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_rios

,Id,Values.Code,Values.Name,Values.HydroRegion,Values.Status
0,Sistema,ALICBOGO,BOGOTA N.R.,CENTRO,ACTIVO
1,Sistema,ALTOANCH,ALTOANCHICAYA,VALLE,ACTIVO
2,Sistema,AMOYAMOY,AMOYA,CENTRO,ACTIVO
3,Sistema,BAJOANCH,DIGUA,VALLE,ACTIVO
4,Sistema,BETAMAG1,BETANIA CP,CENTRO,ACTIVO
5,Sistema,BOCATENC,CONCEPCION,ANTIOQUIA,ACTIVO
6,Sistema,CAL1CALM,CALIMA,VALLE,ACTIVO
7,Sistema,CAMECAMP,CAMPOALEGRE,CALDAS,ACTIVO
8,Sistema,CAMECHIN,CHINCHINA,CALDAS,ACTIVO
9,Sistema,CLLRPORC,CARLOS LLERAS,ANTIOQUIA,ACTIVO


In [23]:
listado_rios.to_json('./data/listado_rios.json', orient='records', lines=True)

### Listado Embalses

In [24]:
request = {
    "MetricId": "ListadoEmbalses",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_embalses = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_embalses

,Id,Values.Code,Values.Name,Values.HydroRegion,Values.Status
0,Sistema,AGREGADO,AGREGADO BOGOTA,CENTRO,ACTIVO
1,Sistema,ALTOANCH,ALTOANCHICAYA,VALLE,ACTIVO
2,Sistema,BETANIA,BETANIA,CENTRO,ACTIVO
3,Sistema,CALIMA1,CALIMA1,VALLE,ACTIVO
4,Sistema,CHUZA,CHUZA,ORIENTE,ACTIVO
5,Sistema,ELQUIMBO,EL QUIMBO,CENTRO,ACTIVO
6,Sistema,ESMERALD,ESMERALDA,ORIENTE,ACTIVO
7,Sistema,FLORIDA2,FLORIDA II,RIOS ESTIMADOS,ACTIVO
8,Sistema,GUAVIO,GUAVIO,ORIENTE,ACTIVO
9,Sistema,GUAVIO,GUAVIO,ORIENTE,ACTIVO


In [25]:
listado_embalses.to_json('./data/listado_embalses.json', orient='records', lines=True)

### Listado Empresas del Sistema Electrico

In [27]:
request = {
    "MetricId": "ListadoAgentes",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_agentes = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_agentes

,Id,Values.Code,Values.Name,Values.Activity,Values.OperStartdate,Values.State
0,Sistema,GREC,GRUPO ENERGÉTICO SA ESP,COMERCIALIZACIÓN,2024-05-15,OPERACION
1,Sistema,SDEG,SOLAR EL EDÉN S.A.S. E.S.P.,GENERACIÓN,2024-05-07,OPERACION
2,Sistema,CPSG,EL CAMPANO SOLAR S.A.S. E.S.P.,GENERACIÓN,2024-04-30,OPERACION
3,Sistema,SHAG,OPERADORA SHANGRI-LA S.A.S E.S.P,GENERACIÓN,2024-04-30,OPERACION
4,Sistema,CSMG,SOLAR VILLAVIEJA S.A.S. E.S.P.,GENERACIÓN,2024-04-29,OPERACION
...,...,...,...,...,...,...
612,Sistema,CTGC,EMPRESAS MUNICIPALES DE CARTAGO S.A. E.S.P.,COMERCIALIZACIÓN,1995-07-20,INACTIVO
613,Sistema,CTGD,EMPRESAS MUNICIPALES DE CARTAGO S.A. E.S.P.,DISTRIBUCIÓN,1995-07-20,INACTIVO
614,Sistema,EPPC,EMPRESAS PUBLICAS DE PEREIRA S.A. E.S.P.,COMERCIALIZACIÓN,1995-07-20,INACTIVO
615,Sistema,EPMT,EMPRESAS PUBLICAS DE MEDELLIN E.S.P.,TRANSPORTE,1995-01-01,OPERACION


In [29]:
listado_agentes.to_json('./data/listado_empresas_agentes.json', orient='records', lines=True)

### Listado de Autogeneradores

In [32]:
request = {
    "MetricId": "ListadoAGPE",
    "Entity": "Agente"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_autogeneradores = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_autogeneradores

,Id,Values.AgpeID,Values.Name,Values.CompanyCode,Values.ComZone,Values.VoltageLevel,Values.EnerSource,Values.InstalledCapacity
0,Agente,Agp00001,PROMOTORA RD LTDA,CMMC,CARIBE MAR,2,RAD SOLAR,9.000000
1,Agente,Agp00002,ORGANIZACION TERPEL S A,EPSC,VALLE DEL CAUCA,2,GAS,16.011109
2,Agente,Agp00003,ANDREA BARRIOS,CNSC,NORTE DE SANTANDER,1,RAD SOLAR,5.000000
3,Agente,Agp00004,JUAN JOSE SANTA VALLA,CMMC,CARIBE MAR,2,RAD SOLAR,10.000000
4,Agente,Agp00005,LUIS ALFONSO VILLAMIL VARGAS,CSSC,CARIBE SOL,1,RAD SOLAR,3.000000
...,...,...,...,...,...,...,...,...
8787,Agente,Grp0988,AGPE FUNDACION OFTALMOLOGICA,BIAC,NARIÑO,2,RAD SOLAR,20.000000
8788,Agente,Grp0993,AGPE RAMON GIRALDO GONZALEZ,SCEC,CARIBE MAR,1,RAD SOLAR,27.000000
8789,Agente,Unt0499,AGPE - ECOPETROL LA HORMIGA,GECG,BAJO PUTUMAYO,3,GAS,1000.000000
8790,Agente,Unt0514,AGPE ENTREPALMAS,EPSG,META,2,BIOGAS,500.000000


## Extracción de Variables de Generación

### Generación del Sistema por Hora

In [43]:
date_dict = generate_date_dict(2010, 2010, 4)
metric = "Gene"
num_metric = 0
df_generacion_sistema = extract_metric(date_dict,metric,num_metric)
df_generacion_sistema

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30


,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,5609982.23000,5296750.71000,4919467.58000,4638776.22000,4442074.76000,4279026.64000,3827254.47000,3832573.95000,...,4391372.59000,4365640.95000,4605721.12000,6021430.06000,6527391.35000,6261393.45000,5776398.57000,5211439.50000,4714078.01000,2010-01-01
1,Sistema,Sistema,4443442.83000,4238022.31000,4153685.87000,4103574.79000,4153986.60000,4252501.43000,4185993.85000,4479587.60000,...,5397357.88000,5317214.52000,5482854.99000,6895377.37000,7356071.75000,7109606.16000,6611477.08000,5931707.62000,5211930.81000,2010-01-02
2,Sistema,Sistema,4771417.74000,4520041.29000,4364500.03000,4210941.04000,4205070.55000,4312551.86000,4156670.92000,4331637.24000,...,5015866.10000,4981069.04000,5169045.72000,6637669.36000,7126930.30000,6924827.71000,6432570.74000,5752050.08000,5135978.48000,2010-01-03
3,Sistema,Sistema,4705250.02000,4493314.06000,4399068.67000,4395569.56000,4475996.61000,4777750.57000,4911096.21000,5322214.49000,...,6618343.03000,6458778.34000,6529860.50000,7819390.86000,8232402.15000,7884258.05000,7197982.57000,6288573.58000,5535400.79000,2010-01-04
4,Sistema,Sistema,5171964.92000,4939629.25000,4840107.60000,4774763.87000,4949684.01000,5195571.83000,5362721.47000,5742326.06000,...,6778398.35000,6715045.20000,6745497.24000,8023899.14000,8336888.23000,7929022.67000,7243496.48000,6378988.75000,5737144.04000,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Sistema,Sistema,5111450.17000,4930718.02000,4828527.23000,4832894.82000,5104736.60000,5831680.58000,6099541.61000,6384431.65000,...,7308981.42000,7268627.40000,7362446.47000,8407732.21000,8656321.39000,8220035.10000,7581677.66000,6649667.38000,5778776.50000,2010-04-26
116,Sistema,Sistema,5338372.69000,5115268.14000,5024674.77000,5036287.36000,5318025.33000,6091092.80000,6231040.80000,6452998.32000,...,7477008.15000,7453863.73000,7493518.04000,8463531.60000,8665651.95000,8204151.42000,7559778.25000,6590307.63000,5695970.25000,2010-04-27
117,Sistema,Sistema,5244268.68000,5090991.91000,4944147.95000,5021515.67000,5215423.00000,6012148.04000,6192910.71000,6463842.30000,...,7369757.90000,7322446.44000,7453349.17000,8405845.10000,8598147.33000,8126827.34000,7498887.19000,6608588.17000,5730363.19000,2010-04-28
118,Sistema,Sistema,5300404.42000,5107992.90000,4990472.84000,5007353.67000,5285903.53000,5988429.50000,6220398.08000,6465528.83000,...,7273977.02000,7258335.66000,7324254.11000,8357989.56000,8644476.93000,8218848.97000,7508293.47000,6598772.71000,5808793.59000,2010-04-29
